### Building A chatbot

In this we will cover an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remeber previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation.There are several other related concepts that you may be looking for:

* Convrsational RAG: Enable a chatbot experience over an external source of data.

* Agents: Build a chatbot that can take actions


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key= os.getenv('GROQ_API_KEY')


In [3]:
from langchain_groq import ChatGroq
model= ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001475E85A660>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001475E85B230>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi My name is Neha and I am a chief AI engineer")])



AIMessage(content="Hello Neha, it's nice to meet you! That's a fascinating role. \n\nWhat kind of AI projects are you working on? I'm always eager to learn more about the innovative things people are doing with artificial intelligence.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 22, 'total_tokens': 75, 'completion_time': 0.096363636, 'prompt_time': 0.00132279, 'queue_time': 0.256937732, 'total_time': 0.097686426}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--e015c7be-642e-4f64-a8c8-2e9962793a88-0', usage_metadata={'input_tokens': 22, 'output_tokens': 53, 'total_tokens': 75})

In [5]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi My name is Neha and I am a chief AI engineer"),
        AIMessage(content="Hello Neha, it's nice to meet you!\n\nThat's a fantastic role. As a large language model, I'm always interested in learning more about the work that AI engineers do.\n\nWhat kind of projects are you currently working on?  \n\nI hope you have a productive day!\n"),
        HumanMessage(content="What's my name and what do i do?")
    ]
)
# first of all we are fieding the information and asking it again toi check whether it is remebering my name. hence it is saving the message history

AIMessage(content="You said your name is Neha and that you are a chief AI engineer. 😊  \n\nIs there anything else you'd like to tell me about yourself or your work? I'm always eager to learn more! \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 107, 'total_tokens': 156, 'completion_time': 0.089090909, 'prompt_time': 0.002748278, 'queue_time': 0.257645353, 'total_time': 0.091839187}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2a61f841-e473-43a1-8632-beb827e2db08-0', usage_metadata={'input_tokens': 107, 'output_tokens': 49, 'total_tokens': 156})

### Message History

We can use a Message history class to wrap our model and make it stateful. this will keep track of inputs and outputs of the model and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# for understanding the history of the particular user we define this function so that the llm can understand when used by multiple user at a time

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)


In [7]:
config={"configurable":{"session_id":"chat1"}}

In [8]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Neha and I am a chief AI engineer")],
    config=config
)
response.content

"Hi Neha, it's nice to meet you!\n\nThat's a fascinating role. What kind of projects are you working on as a Chief AI Engineer?  \n\nI'm always eager to learn more about how AI is being used in different fields.\n"

In [9]:
response=with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config=config
)
response.content

'Your name is Neha.  You told me at the beginning of our conversation. 😊 \n\n'

#### changing the config (session_id) to check whether it remenbers the name or not

In [10]:
config1={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config=config1
)
response.content

"Your name is Neha!  \n\nIs there something specific you'd like to talk about regarding your name? \n"

In [11]:
# in abpve we are getting the response because the session_id is same
# let's change it and then check
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to remember it! 😊\n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Sharma")],
    config=config1
)
response.content

'Nice to meet you, Sharma! \n\nIs there anything I can help you with today? 😊\n'

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config=config1
)
response.content

'Your name is Sharma!  I remember now. 😊  \n\nDo you have any other questions for me? \n'

### Prompt templates

These help to turn raw user information into a fromat that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. first let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [14]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt=ChatPromptTemplate(
    [
        ("system","You are a helpful AI assistant. Answer all the questions to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")

    ]
)

chain=prompt|model

In [15]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is neha")]})

AIMessage(content="Hi Neha, it's nice to meet you!  \n\nHow can I help you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 32, 'total_tokens': 59, 'completion_time': 0.049090909, 'prompt_time': 0.001539569, 'queue_time': 0.256520843, 'total_time': 0.050630478}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--0c12fa54-f628-48af-b173-c88819277fa6-0', usage_metadata={'input_tokens': 32, 'output_tokens': 27, 'total_tokens': 59})

In [16]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [17]:
config= {"configurable": {"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Neha")],
    config=config
)
response.content

"Hi Neha, it's nice to meet you! \n\nHow can I help you today? 😊  \n\n"

In [29]:
#Add more complexity

prompt=ChatPromptTemplate(
    [
        ("system","You are a helpful AI assistant. Answer all the questions to the best of your ability in {bhasha}"),
        MessagesPlaceholder(variable_name="messages")

    ]
)
chain=prompt|model


In [30]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Neha")], "bhasha":"Hindi"})
response.content

'नमस्ते, नीहा! \n\nमुझे आपकी मदद करने में खुशी हो रही है। आप मुझसे क्या पूछना चाहती हैं? 😊 \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [31]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [33]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi i am neha")],"bhasha":"hindi"},
    config=config
)
response.content

# so for input messages key we can set it to any thing but it should be similar in all prompt with msg and invoke

'नमस्ते नीहा! 👋 \n\nआप कैसे हैं? 😊 \n'

### Manage the Conversation History

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

''trim_messages'' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [36]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer=trim_messages(
    max_tokens=70,
    strategy="last" , # from last it is focusing only on the last conversation we had
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="You are a helpful assistant that adapts to the user's tone and responds appropriately."),

    HumanMessage(content="Tell me the benefits of daily exercise."),
    AIMessage(content="• Improves cardiovascular health\n• Boosts mood\n• Increases energy levels"),

    HumanMessage(content="Can you explain photosynthesis in a simple way?"),
    AIMessage(content="Sure! Plants use sunlight, water, and carbon dioxide to make food and release oxygen."),

    HumanMessage(content="Now explain it like a Shakespearean poet."),
    AIMessage(content="Lo, the green leaf, with sun so bright, doth feast on air and drink the light."),

    HumanMessage(content="List top 3 programming languages in 2025 in JSON format."),
    AIMessage(content='{"languages": ["Python", "JavaScript", "Rust"]}'),

    HumanMessage(content="I’m feeling down and demotivated today."),
    AIMessage(content="You've got this! Every step forward, no matter how small, is progress. Keep going 💪"),

    HumanMessage(content="Okay, but now talk like a pirate!"),
    AIMessage(content="Arrr matey! Chase yer dreams like a ship chases treasure across the seven seas!"),

    HumanMessage(content="One-line answer: What's the capital of Australia?"),
    AIMessage(content="Canberra.")
]

trimmer.invoke(messages)


[SystemMessage(content="You are a helpful assistant that adapts to the user's tone and responds appropriately.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Okay, but now talk like a pirate!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Arrr matey! Chase yer dreams like a ship chases treasure across the seven seas!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="One-line answer: What's the capital of Australia?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Canberra.', additional_kwargs={}, response_metadata={})]

In [35]:
# if we have chain then how we are going to apply the trimmer 
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    |model
)

response=chain.invoke(
    {
        "messages":messages + [HumanMessage(content= "How i am feeling today?")],
        "bhasha":"English"
    }  
)
response.content

"As an AI, I can't know how you're feeling. How are *you* feeling today? 😊  \n\n\n\n"

In [37]:
response=chain.invoke(
    {
        "messages":messages + [HumanMessage(content= "For which country i have asked for capital?")],
        "bhasha":"English"
    }  
)

response.content

'Australia 🇦🇺  '

In [38]:
#Let's warp this in the Message History -->
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
    
)
config={"configurable" :{"session_id":"chat5"}}

In [40]:
response=with_message_history.invoke(
    {
        "messages":messages+[HumanMessage(content="What's my name")],
        "bhasha":"english"
    },
    config=config
)
response.content

"I don't have access to any information about you, including your name.  \n\n\n"